## Proyecto final Upgrade HUB

Empezamos cargando las librerías que usaremos a lo largo del proyecto

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stats
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


A continuación, cargamos nuestros datos y haremos un pequeño análisis descriptivo para ver si es necesario procesar los datos

In [ ]:
df_wines = pd.read_csv("wines_SPA.csv")
df_wines.head()

In [ ]:
df_wines.describe()

In [ ]:
df_wines.isnull().sum()

In [ ]:
df_wines.head(20)

In [ ]:
df_wines["wine"].mode()

In [ ]:
null_percentage = df_wines.isnull().sum() / len(df_wines) * 100


import seaborn as sns

null_percentage = df_wines.isnull().sum() / len(df_wines) * 100

plt.figure(figsize=(10, 6))
plt.title('Valores nulos en el dataset')
sns.set_palette("coolwarm")
sns.barplot(x=null_percentage.index, y=null_percentage.values)
plt.xticks(rotation=45)
plt.xlabel('Columnas',color='white', fontsize=15)
plt.ylabel('Porcentaje de valores nulos',color='black', fontsize=15)
plt.title('Valores nulos en el dataset', color='black', fontsize=15)
# Set the style to whitegrid
sns.set_style("whitegrid")

plt.gca().set_facecolor('none')
plt.savefig("valores_nulos.png")  # Guardar la imagen
plt.show()

In [ ]:
filtered_df = df_wines[df_wines['acidity'].isnull() | df_wines['body'].isnull()]
filtered_df

Viendo que es un número bajo de valores nulos y que no son facilmente reemplazables (debido a que puede afectar al estudio) procederemos a eliminarlos

In [ ]:
# eliminar filas que tengan body y acidity nulos
df_wines_cleanbodyacid = df_wines.drop(filtered_df.index)

In [ ]:
df_wines_cleanbodyacid.isnull().sum()   

In [ ]:
df_wines_cleanbodyacid.head()

Al cargar los datos en power BI descubrimos que habia valores nulos camuflados en la base de datos donde en vez de poner el año ponia "N.V.", asi que procederemos a sustituirlo por valores nulos y posteriormente eliminarlos

In [ ]:
filtered_df_YEAR = df_wines_cleanbodyacid[df_wines_cleanbodyacid['year'] == 'N.V.']
filtered_df_YEAR

In [ ]:
df_wines_cleanbodyacid["year"] = df_wines_cleanbodyacid["year"].replace("N.V.", np.nan)

In [ ]:
df_wines_cleanbodyacid.isnull().sum()

In [ ]:
df_wines_cleanbodyacidyear = df_wines_cleanbodyacid.dropna()

In [ ]:
df_wines_cleanbodyacidyear.isnull().sum()

Por último creamos un nuevo csv con la base de datos limpia que sera con el que trabajaremos a partir de ahora

In [ ]:
df_wines_cleanbodyacidyear.to_csv('dfclean.csv', index=False)

In [45]:
dfclean = pd.read_csv('dfclean.csv')
dfclean.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [ ]:

sns.set(style="whitegrid")
def create_boxplot(df, column, title):
    plt.figure(figsize=(10, 6))
    ax = sns.boxplot(x=df[column])
    ax.set_xticklabels(df['type'], rotation=45, ha='right')
    plt.title(title)
    plt.show()
create_boxplot(dfclean, 'num_reviews', 'Boxplot de Número de Reseñas')
create_boxplot(dfclean, 'price', 'Boxplot de Precio')
create_boxplot(dfclean, 'body', 'Boxplot de Cuerpo')

Resultados de la detección de outliers

	1.	Rating (calificación) Total de outliers detectados: 596. Estos outliers pueden indicar vinos que recibieron calificaciones extremadamente altas o bajas en comparación con otros.
	2.	Número de reseñas Total de outliers detectados: 1622: sugiere que algunos vinos han recibido un número de reseñas significativamente mayor o menor que otros. 
	3.	Precio Total de outliers detectados: 436
	4.	Cuerpo Total de outliers detectados: 34
	5.	Acidez Total de outliers detectados: 294


## Modelo de clasificación random forest

In [47]:
df= pd.read_csv("dfclean.csv") 

In [ ]:
## Hacien uso de la funcion label encoder codificamos las variables
label_encoders = {}
for column in ['wine', 'country', 'region', 'type']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [ ]:
#definimos la variable objetivo, bodega en nuestro caso. 
X = df.drop(columns='winery')
y = df['winery']

In [ ]:
#generamos los split de entrenamiento y testeo en proporcion 80 20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Creamos el modelo random forest usando la libreria scikit-learn, fijamos el numero de arboles en 100 y generamos una semilla para asegurar que sea reproducible
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
#a continuacion enttrenamos el modelos usando los splits del paso anterior
rf_model.fit(X_train, y_train)

In [ ]:
#ejecutamos el modelo y usando sus predicciones calculamos los estadisticos para ver como de preciso es
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
#Creamos un array en el que asignamos a cada variable su valor de varianza explicada
feature_importances = rf_model.feature_importances_
features = X.columns

#creamos un dataframe con los datos del paso anterior
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

#ordenamos el dataframe por orden de importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

#lo graficamos y guardamos la imagen
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Importancia de las Características en el Modelo de Random Forest')
plt.xlabel('Importancia')
plt.ylabel('Característica')
plt.show()

## Probamos el modelo que luego añadiremos al streamlit, añadiendo entradas por teclado para probar las predicciones

In [ ]:
df = pd.read_csv("dfclean.csv")

# Se convierten las variables categóricas a numéricas
label_encoders = {}
for column in ['wine', 'country', 'region', 'type']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# se define la variable objetivo en nuestro caso hemos elegido la bodega
X = df.drop(columns='winery')
y = df['winery']

# Se crean el conjunto de train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# obtenemos los estadisticos del modelo
y_pred = rf_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


def predict_winery(wine, year, rating, num_reviews, country, region, price, wine_type, body, acidity):
    #se codifican las variables
    wine = label_encoders['wine'].transform([wine])[0]
    country = label_encoders['country'].transform([country])[0]
    region = label_encoders['region'].transform([region])[0]
    wine_type = label_encoders['type'].transform([wine_type])[0]
    
    
    new_data = pd.DataFrame({
        'wine': [wine],
        'year': [year],
        'rating': [rating],
        'num_reviews': [num_reviews],
        'country': [country],
        'region': [region],
        'price': [price],
        'type': [wine_type],
        'body': [body],
        'acidity': [acidity]
    })

    # Realizar la predicción
    prediction = rf_model.predict(new_data)
    return prediction[0]

# Se piden los valores por pantalla
print("Por favor ingrese los valores del vino para predecir la bodega:")
wine = input("Tipo de Vino (e.g., Tinto): ")
year = (input("Año del Vino (e.g., 2013): "))
rating = (input("Rating del Vino (e.g., 4.9): "))
num_reviews = int(input("Número de Reviews (e.g., 58): "))
country = input("País del Vino (e.g., Espana): ")
region = input("Región del Vino (e.g., Toro): ")
price = (input("Precio del Vino (e.g., 995.0): "))
wine_type = input("Tipo de Vino (e.g., Toro Red): ")
body = (input("Cuerpo del Vino (e.g., 5.0): "))
acidity = (input("Acidez del Vino (e.g., 3.0): "))

# Realizar la predicción
prediction = predict_winery(wine, year, rating, num_reviews, country, region, price, wine_type, body, acidity)
print(f"La predicción de la bodega es: {prediction}")